In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score


In [2]:
# Assuming df is already defined and read from 'data.csv'
df = pd.read_csv('all_seasons.csv')

# Select multiple columns to stack
columns_to_stack = ['gp', 'pts', 'reb', 'net_rating', 'ts_pct', 'net_rating', 'ast_pct', 'age']

# Extract these columns as a 2D NumPy array
data_to_stack = df[columns_to_stack].to_numpy()

# Standardization using StandardScaler
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data_to_stack)

# Apply PCA
n_components = 4
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(data_standardized)

# Convert the PCA result to a DataFrame
X = pd.DataFrame(X_pca, columns=[f'PCA {i+1}' for i in range(n_components)])
print(X)

# Extract the target variable
y = df['usg_pct'].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Verify the shapes of the split data
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


          PCA 1     PCA 2     PCA 3     PCA 4
0      0.264562  0.380113  0.722649 -1.578952
1      0.822315  2.292514 -0.418262  0.167787
2      0.156114 -0.825565 -0.478057  0.078284
3     -0.288905 -0.143887  0.312930  0.587398
4      1.654264 -0.365432 -0.869909 -0.452555
...         ...       ...       ...       ...
12839 -4.420308 -2.086314  0.563676 -0.010408
12840  2.833356 -0.026164 -0.838516 -1.202145
12841 -1.392419 -0.791279 -1.311929  0.051785
12842 -0.332588 -0.334212 -1.833260 -0.007769
12843 -0.108591 -0.351896 -0.088934  1.536899

[12844 rows x 4 columns]
x_train shape: (10275, 4)
x_test shape: (2569, 4)
y_train shape: (10275,)
y_test shape: (2569,)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network using TensorFlow/Keras
def create_neural_network(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create the neural network
input_dim = x_train.shape[1]
model = create_neural_network(input_dim)
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Training the neural network
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test data
loss, mae = model.evaluate(x_test, y_test, verbose=1)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


Epoch 1/100


257/257 [==============================] - 2s 3ms/step - loss: 0.0037 - mae: 0.0427 - val_loss: 0.0024 - val_mae: 0.0344
Epoch 2/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0025 - mae: 0.0353 - val_loss: 0.0027 - val_mae: 0.0357
Epoch 3/100
257/257 [==============================] - 1s 3ms/step - loss: 0.0023 - mae: 0.0350 - val_loss: 0.0022 - val_mae: 0.0318
Epoch 4/100
257/257 [==============================] - 1s 2ms/step - loss: 0.0021 - mae: 0.0335 - val_loss: 0.0024 - val_mae: 0.0331
Epoch 5/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0020 - mae: 0.0323 - val_loss: 0.0023 - val_mae: 0.0325
Epoch 6/100
257/257 [==============================] - 1s 2ms/step - loss: 0.0021 - mae: 0.0332 - val_loss: 0.0022 - val_mae: 0.0323
Epoch 7/100
257/257 [==============================] - 1s 2ms/step - loss: 0.0019 - mae: 0.0323 - val_loss: 0.0036 - val_mae: 0.0351
Epoch 8/100
257/257 [==============================] - 1s 2ms/step 

In [8]:
from sklearn.metrics import r2_score
print(f'R² score for training data: {r2_train:.4f}')
print(f'R² score for testing data: {r2_test:.4f}')

R² score for training data: 0.4457
R² score for testing data: 0.4121


In [7]:
from sklearn.metrics import accuracy_score
# Predict using the trained model
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Convert predictions to class labels
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

# Calculate accuracy score
train_accuracy = accuracy_score(y_train, y_train_pred_classes)
test_accuracy = accuracy_score(y_test, y_test_pred_classes)

print(f'Accuracy score for training data: {train_accuracy:.4f}')
print(f'Accuracy score for testing data: {test_accuracy:.4f}')


81/81 [==============================] - 0s 934us/step


ValueError: Classification metrics can't handle a mix of continuous and binary targets